In [ ]:
%pip install DBcm --upgrade

In [ ]:
import DBcm

In [ ]:
db_details = "SwimDB.sqlite3"

In [ ]:
with DBcm.UseDatabase(db_details) as db:
    db.execute("pragma table_list")
    resuts = db.fetchall()

In [ ]:
resuts

In [ ]:
SQL_SWIMMERS = """
    create table if not exists swimmers(
        id integer not null primary key autoincrement,
        name varchar(32) not null,
        age integer not null
    )
"""

SQL_EVENTS = """
    create table if not exists events(
        id integer not null primary key autoincrement,
        distance varchar(16) not null,
        stroke varchart(16) not null
    )
"""

SQL_TIMES = """
    create table if not exists times(
        swimmer_id integer not null,
        event_id integer not null,
        time varchart(16),
        ts timestamp default current_timestamp
    )
"""

with DBcm.UseDatabase(config=db_details) as db:
    db.execute(SQL_SWIMMERS)
    db.execute(SQL_EVENTS)
    db.execute(SQL_TIMES)
    db.execute("pragma table_list")
    result = db.fetchall()

In [ ]:
result

## Populate tables

In [ ]:
import os

FOLDER = "swimdata/"

files = os.listdir(FOLDER)
files.remove(".DS_Store")

In [ ]:
file_name = files[0]
name, age, distance, stroke = file_name.removesuffix(".txt").split("-")

In [ ]:
SQL_INSERT_SWIMMER = """
    insert into swimmers
    (name, age)
    values
    (?,?)
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_INSERT_SWIMMER, (name, age))
    db.execute("pragma table_list")
    result = db.fetchall()
    
result

In [ ]:
SQL_SELECT_SWIMMER = """
    select *
    from swimmers
    where name = ? and age = ?;
"""

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SELECT_SWIMMER, (name, age))
    result = db.fetchall()

result
if result:
    print("true")
else:
    print("false")

In [ ]:
SQL_DELETE = "delete from times;"

with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_DELETE)
    db.execute(SQL_SELECT_SWIMMER, (name, age))
    result = db.fetchall()

result

In [ ]:

with DBcm.UseDatabase(db_details) as db:
    for file_name in files:
        name, age, *_ = file_name.removesuffix(".txt").split("-")
        db.execute(SQL_SELECT_SWIMMER, (name, age))
        result = db.fetchall()
        if result:
            continue
        db.execute(SQL_INSERT_SWIMMER, (name, age))

In [ ]:
SQL_INSERT_EVENT = """
    insert into events
    (distance, stroke) 
    values
    (?, ?);
"""

SQL_SELECT_EVENT = """
    select *
    from events
    where distance = ? and stroke = ?;
"""

In [ ]:
with DBcm.UseDatabase(db_details) as db:
    for file_name in files:
        *_, distance, stroke = file_name.removesuffix(".txt").split("-")
        db.execute(SQL_SELECT_EVENT, (distance, stroke))
        result = db.fetchall()
        if result:
            continue
        db.execute(SQL_INSERT_EVENT, (distance, stroke))

In [ ]:
SQL_GET_EVENT_ID = """
    select id
    from events
    where distance = ? and stroke = ?;
"""

SQL_GET_SWIMMER_ID = """
    select id
    from swimmers
    where name = ? and age = ?;
"""

SQL_INSERT_TIME = """
    insert into times
    (swimmer_id, event_id, time)
    values
    (?,?,?);
"""

In [ ]:
from swimclub import DATA_FOLDER

In [ ]:
with DBcm.UseDatabase(db_details) as db:
    for file_name in files:
        name, age, distance, stroke = file_name.removesuffix(".txt").split("-")

        db.execute(SQL_GET_SWIMMER_ID, (name, age))
        result = db.fetchone()
        swimmer_id = result[0]

        db.execute(SQL_GET_EVENT_ID, (distance, stroke))
        result = db.fetchone()
        event_id = result[0]

        with open(DATA_FOLDER + file_name, "r") as file:
            lines = file.readlines()
        times = lines[0].strip().split(",")

        for time in times:
            db.execute(SQL_INSERT_TIME, (swimmer_id, event_id, time))